### First, we set up our environment (tokens and such) in order to be able to create a slice. This is not the focus of this tutorial.

In [1]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net


In [2]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTIMBXMMZTQMJYHFRDGNDFMM2TCZRXGA4DQOBTHEYDAZBTGM2TQYR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRVGE2TIMRTHA4DQOBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTKZTDMVRWMNLBMMZWIZBUGU4WIODEMRSTSZTCGVRTIOJUMZSDMNZ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRVGE2DAMZTGA3DAMZGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


In [3]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

### Get new Fabric Identity Token and update Fabric Refresh Token

In [4]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print()


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTONBQMU2DCOJQMFQTMMJWGU2GIOLEG43DCZBZGJRTCYZSGVTDSYZ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRVGE2TIMRZGM4TKNZGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)




# Tutorial starts here

### We are going to be creating slices that contain one node each.

### We need to specify the resources (number of cores, amount of ram and amount of disk space) that we want to allocate to our node.

### We can do that in two ways:
 - Using Capacities()
 - Using capacity hints.

### Let's create our first slice that contains one node. We will use `Capacities()` to specify the resources that we want to allocate.

### The line `cap.set_fields(core=2, ram=8, disk=10)` specifies that we want to reserve a node with 2 cores, 8GB of RAM and 10GB of disk.

In [5]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='UKY')

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=8, disk=10)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='JupyterSlice1', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

Response Status Status.OK
Reservations created [{
    "graph_node_id": "758f3d51-6389-4290-824f-2384f64cecbf",
    "join_state": "None_",
    "lease_end": "2021-07-02 15:44:58",
    "reservation_id": "bc35e413-9821-4709-9d98-623c86af37e0",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "efbc0d18-c881-4bfb-ab1d-a2033325ae04"
}]


In [6]:
# Set the Slice ID from output of the above command
slice_id=reservations[0].slice_id

### We have issued the request to create the slice with the node. Let's query our slices.

In [9]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

Response Status Status.OK
Slices [{
    "graph_id": "5b9c7977-8d59-480f-b732-d64cd4ba110b",
    "slice_id": "efbc0d18-c881-4bfb-ab1d-a2033325ae04",
    "slice_name": "JupyterSlice1",
    "slice_state": "StableOK"
}]


### Our slice is ready. Now we can query the slivers (nodes, etc...) that are inside the slice.

In [10]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Sliver {}".format(slivers))

Response Status Status.OK
Sliver [{
    "allocated_capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 8}",
    "allocated_labels": "{\"instance\": \"instance-00000174\", \"instance_parent\": \"uky-w1\"}",
    "capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 8}",
    "capacity_hints": "{\"instance_type\": \"fabric.c2.m8.d10\"}",
    "graph_node_id": "758f3d51-6389-4290-824f-2384f64cecbf",
    "join_state": "NoJoin",
    "lease_end": "2021-07-02 15:44:58",
    "management_ip": "128.163.179.33",
    "name": "n1",
    "reservation_id": "bc35e413-9821-4709-9d98-623c86af37e0",
    "reservation_state": "Active",
    "resource_type": "VM",
    "site": "UKY",
    "slice_id": "efbc0d18-c881-4bfb-ab1d-a2033325ae04"
}]


### Let's look carefully at the "allocated_capacities" variable.

In [11]:
print(slivers[0].allocated_capacities)

{ core: 2 , ram: 8 G, disk: 10 G}


### It says that our node has 2 cores, 8GB of RAM and 10GB of disk space, which is what we requested.

### Now let's delete the slice.

<!-- <hr> -->
<!-- <hr> -->

In [12]:
slice_manager.delete(slice_id=slice_id)

(<Status.OK: 1>, None)

<hr>
<hr>

### Now let's try something else. Let's try to request 2 cores, 8GB of RAM, and 50GB of disk space.

### Again, we are going to use `cap.set_fields(core=2, ram=8, disk=50)`.

In [13]:
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='UKY')

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=8, disk=50)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='JupyterSlice2', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

Response Status Status.OK
Reservations created [{
    "graph_node_id": "b1855d04-1b84-49b2-a9b4-8684ddeb15d8",
    "join_state": "None_",
    "lease_end": "2021-07-02 15:46:09",
    "reservation_id": "d84b09f5-2c33-482d-baac-817d2d4cc2db",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "29c9342c-d937-4b41-94ad-7a75efe39135"
}]


In [14]:
# Set the Slice ID from output of the above command
slice_id=reservations[0].slice_id

### We have issued the request. Let's check our slices again.

In [17]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

Response Status Status.OK
Slices [{
    "graph_id": "407fd181-0e07-4965-9de2-202a37e63280",
    "slice_id": "29c9342c-d937-4b41-94ad-7a75efe39135",
    "slice_name": "JupyterSlice2",
    "slice_state": "StableOK"
}]


### And then query our slivers.

In [18]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Sliver {}".format(slivers))

Response Status Status.OK
Sliver [{
    "allocated_capacities": "{\"core\": 2, \"disk\": 100, \"ram\": 8}",
    "allocated_labels": "{\"instance\": \"instance-00000175\", \"instance_parent\": \"uky-w1\"}",
    "capacities": "{\"core\": 2, \"disk\": 50, \"ram\": 8}",
    "capacity_hints": "{\"instance_type\": \"fabric.c2.m8.d100\"}",
    "graph_node_id": "b1855d04-1b84-49b2-a9b4-8684ddeb15d8",
    "join_state": "NoJoin",
    "lease_end": "2021-07-02 15:46:09",
    "management_ip": "128.163.179.25",
    "name": "n1",
    "reservation_id": "d84b09f5-2c33-482d-baac-817d2d4cc2db",
    "reservation_state": "Active",
    "resource_type": "VM",
    "site": "UKY",
    "slice_id": "29c9342c-d937-4b41-94ad-7a75efe39135"
}]


### And then we can look at the `allocated_capacities` variable.

In [19]:
print(slivers[0].allocated_capacities)

{ core: 2 , ram: 8 G, disk: 100 G}


### We can see that we were allocated 2 cores, 8GB of ram, but 100GB of disk space instead of 50GB.

### The reason for this is that we have discrete "capacity hints". The node can only be an instance of one of those capacity hints.

### _See the very last cell in this notebook for the complete list of available capacity hints._

### This is an exerpt of the available capacity hints.

    "fabric.c16.m64.d10": {"core":16, "ram":64, "disk": 10},
    "fabric.c32.m128.d10": {"core":32, "ram":128, "disk": 10},
    "fabric.c1.m4.d100": {"core":1, "ram":4, "disk": 100},
    "fabric.c2.m8.d100": {"core":2, "ram":8, "disk": 100},
    "fabric.c4.m16.d100": {"core":4, "ram":16, "disk": 100},

### We can see that the disk space can only be 10GB or 100GB. So when we requested 50GB, it was rounded up to 100GB.

<!-- <hr> -->
<!-- <hr> -->

### Now let's delete the slice.

In [21]:
slice_manager.delete(slice_id=slice_id)

(<Status.OK: 1>, None)

<hr>
<hr>

### Finally, we can directly set the resources that we need using a "capacity hint" string. _Please see the very last cell in this notebook for the complete list of available capacity hints._

### We can set the needed resources like so:

### `capacity_hints=CapacityHints().set_fields(instance_type='fabric.c2.m8.d10')`.

### This would reserve a node with 2 processor cores, 8GB of memory and 10GB of disk space.
 - The number next to the `c` is the number of cores.
 - The number next to the `m` is the amount of memory in GB.
 - The number next to the `d` is the amount of disk space in GB.

### We can pick any capacity hint string from the list.

In [22]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, CapacityHints
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='UKY')

# Set Properties
n1.set_properties(capacity_hints=CapacityHints().set_fields(instance_type='fabric.c2.m8.d10'), image_type='qcow2', image_ref='default_ubuntu_20')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='JupyterSlice3', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

Response Status Status.OK
Reservations created [{
    "graph_node_id": "3b527901-a5fc-4682-ab95-f56fc8c0dc8a",
    "join_state": "None_",
    "lease_end": "2021-07-02 15:47:27",
    "reservation_id": "23d1faf0-7961-49ac-8925-3c9e568da9f6",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "188018cd-8684-43fa-9c0f-c7b40b5279a0"
}]


In [23]:
# Set the Slice ID from output of the above command
slice_id=reservations[0].slice_id

### Let's check our slices

In [25]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))

Response Status Status.OK
Slices [{
    "graph_id": "ca0df5e6-cb7e-4d02-a4aa-fcca57e93747",
    "slice_id": "188018cd-8684-43fa-9c0f-c7b40b5279a0",
    "slice_name": "JupyterSlice3",
    "slice_state": "StableOK"
}]


### Now let's query the slivers in JupyterSlice3.

In [26]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Sliver {}".format(slivers))

Response Status Status.OK
Sliver [{
    "allocated_capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 8}",
    "allocated_labels": "{\"instance\": \"instance-00000176\", \"instance_parent\": \"uky-w1\"}",
    "capacities": "{\"core\": 2, \"disk\": 10, \"ram\": 8}",
    "capacity_hints": "{\"instance_type\": \"fabric.c2.m8.d10\"}",
    "graph_node_id": "3b527901-a5fc-4682-ab95-f56fc8c0dc8a",
    "join_state": "NoJoin",
    "lease_end": "2021-07-02 15:47:27",
    "management_ip": "128.163.179.44",
    "name": "n1",
    "reservation_id": "23d1faf0-7961-49ac-8925-3c9e568da9f6",
    "reservation_state": "Active",
    "resource_type": "VM",
    "site": "UKY",
    "slice_id": "188018cd-8684-43fa-9c0f-c7b40b5279a0"
}]


### Let's focus on the `allocated_capacities` varible.

In [27]:
print(slivers[0].allocated_capacities)

{ core: 2 , ram: 8 G, disk: 10 G}


### We can see that we got the resources that we requested.

### Now let's delete the slice.

In [28]:
slice_manager.delete(slice_id=slice_id)

(<Status.OK: 1>, None)

<hr>
<hr>

# Available capacity hints (and their descriptions) below.

    {
      "fabric.c1.m4.d10": {"core":1, "ram":4, "disk": 10},
      "fabric.c2.m8.d10": {"core":2, "ram":8, "disk": 10},
      "fabric.c4.m16.d10": {"core":4, "ram":16, "disk": 10},
      "fabric.c8.m32.d10": {"core":8, "ram":32, "disk": 10},
      "fabric.c16.m64.d10": {"core":16, "ram":64, "disk": 10},
      "fabric.c32.m128.d10": {"core":32, "ram":128, "disk": 10},
      "fabric.c1.m4.d100": {"core":1, "ram":4, "disk": 100},
      "fabric.c2.m8.d100": {"core":2, "ram":8, "disk": 100},
      "fabric.c4.m16.d100": {"core":4, "ram":16, "disk": 100},
      "fabric.c8.m32.d100": {"core":8, "ram":32, "disk": 100},
      "fabric.c16.m64.d100": {"core":16, "ram":64, "disk": 100},
      "fabric.c32.m128.d100": {"core":32, "ram":128, "disk": 100},
      "fabric.c1.m4.d500": {"core":1, "ram":4, "disk": 500},
      "fabric.c2.m8.d500": {"core":2, "ram":8, "disk": 500},
      "fabric.c4.m16.d500": {"core":4, "ram":16, "disk": 500},
      "fabric.c8.m32.d500": {"core":8, "ram":32, "disk": 500},
      "fabric.c16.m64.d500": {"core":16, "ram":64, "disk": 500},
      "fabric.c32.m128.d500": {"core":32, "ram":128, "disk": 500},
      "fabric.c1.m4.d2000": {"core":1, "ram":4, "disk": 2000},
      "fabric.c2.m8.d2000": {"core":2, "ram":8, "disk": 2000},
      "fabric.c4.m16.d2000": {"core":4, "ram":16, "disk": 2000},
      "fabric.c8.m32.d2000": {"core":8, "ram":32, "disk": 2000},
      "fabric.c16.m64.d2000": {"core":16, "ram":64, "disk": 2000},
      "fabric.c32.m128.d2000": {"core":32, "ram":128, "disk": 2000},
      "fabric.c64.m384.d4000": {"core":64, "ram":384, "disk": 4000}
    }